### Data Ingestion to Vector DB pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\pom-m\Desktop\ai\rag-langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf files inside the directory

def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""

    all_documents = []
    pdf_dir = Path(pdf_directory)

    # find all pdf files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process.")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # add source information to metadata
            for doc in documents:
                doc.metadata["source_file"] = pdf_file.name
                doc.metadata["file_type"] = "pdf"
            
            all_documents.extend(documents)
            print(f"✔️ Loaded {len(documents)} pages")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs("../data")

all_pdf_documents

Found 2 PDF files to process.

Processing: ai-report.pdf
✔️ Loaded 71 pages

Processing: NIPS-2017-attention-is-all-you-need-Paper.pdf
✔️ Loaded 11 pages

Total documents loaded: 82


[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-05-02T18:36:15+00:00', 'author': 'U.S. Department of Education', 'moddate': '2023-05-22T15:08:44-04:00', 'title': 'Artificial Intelligence and the Future of Teaching and Learning (PDF)', 'source': '..\\data\\pdf\\ai-report.pdf', 'total_pages': 71, 'page': 0, 'page_label': '1', 'source_file': 'ai-report.pdf', 'file_type': 'pdf'}, page_content='1 \n \n \nArtificial Intelligence \nand the Future of \nTeaching and Learning \nInsights and Recommendations \nMay 2023'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-05-02T18:36:15+00:00', 'author': 'U.S. Department of Education', 'moddate': '2023-05-22T15:08:44-04:00', 'title': 'Artificial Intelligence and the Future of Teaching and Learning (PDF)', 'source': '..\\data\\pdf\\ai-report.pdf', 'total_pages': 71, 'page': 1, 'page_label': '2', 'source_file': 'ai-report.pdf', 'file_type': 'pdf'}, page_content='Art

In [3]:
### Text splitting get into chunks

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [4]:
chunks = split_documents(all_pdf_documents)
chunks

Split 82 documents into 326 chunks

Example chunk:
Content: 1 
 
 
Artificial Intelligence 
and the Future of 
Teaching and Learning 
Insights and Recommendations 
May 2023...
Metadata: {'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-05-02T18:36:15+00:00', 'author': 'U.S. Department of Education', 'moddate': '2023-05-22T15:08:44-04:00', 'title': 'Artificial Intelligence and the Future of Teaching and Learning (PDF)', 'source': '..\\data\\pdf\\ai-report.pdf', 'total_pages': 71, 'page': 0, 'page_label': '1', 'source_file': 'ai-report.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-05-02T18:36:15+00:00', 'author': 'U.S. Department of Education', 'moddate': '2023-05-22T15:08:44-04:00', 'title': 'Artificial Intelligence and the Future of Teaching and Learning (PDF)', 'source': '..\\data\\pdf\\ai-report.pdf', 'total_pages': 71, 'page': 0, 'page_label': '1', 'source_file': 'ai-report.pdf', 'file_type': 'pdf'}, page_content='1 \n \n \nArtificial Intelligence \nand the Future of \nTeaching and Learning \nInsights and Recommendations \nMay 2023'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-05-02T18:36:15+00:00', 'author': 'U.S. Department of Education', 'moddate': '2023-05-22T15:08:44-04:00', 'title': 'Artificial Intelligence and the Future of Teaching and Learning (PDF)', 'source': '..\\data\\pdf\\ai-report.pdf', 'total_pages': 71, 'page': 1, 'page_label': '2', 'source_file': 'ai-report.pdf', 'file_type': 'pdf'}, page_content='Art

### Embedding and VectorStoreDatabase

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
    # def get_embedding_dimension(self) -> int:
    #     """Get the embedding dimension of the model"""
    #     if not self.model:
    #         raise ValueError("Model not loaded")
    #     return self.model.get_sentence_embedding_dimension()

    ### Initialze the embedding manager

In [7]:
embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [8]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
    
    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of Langchain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore = VectorStore()

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [9]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]
texts

['1 \n \n \nArtificial Intelligence \nand the Future of \nTeaching and Learning \nInsights and Recommendations \nMay 2023',
 'Artificial Intelligence and the Future of Teaching and Learning \nMiguel A. Cardona, Ed.D. \nSecretary, U.S. Department of Education \nRoberto J. Rodríguez \nAssistant Secretary, Office of Planning, Evaluation, and Policy Development \nKristina Ishmael \nDeputy Director, Office of Educational Technology \nMay 2023 \n \nExamples Are Not Endorsements \nThis document contains examples and resource materials that are provided for the user’s \nconvenience. The inclusion of any material is not intended to reflect its importance nor is it \nintended to endorse any views expressed or products or services offered. These materials may \ncontain the views and recommendations of various subject matter experts as well as hypertext links, \ncontact addresses, and websites to information created and maintained by other public and private \norganizations. The opinions expressed

In [10]:
### Generate embeddings
embeddings = embedding_manager.generate_embeddings(texts)

### Store in the vector database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 326 texts...


Batches: 100%|██████████| 11/11 [00:09<00:00,  1.20it/s]


Generated embeddings with shape: (326, 384)
Adding 326 documents to vector store...
Successfully added 326 documents to vector store
Total documents in collection: 326
